In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn import datasets, linear_model, preprocessing
import numpy.polynomial.polynomial as poly

import lime
import lime.lime_tabular

import copy

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
df = pd.read_csv("xML Challenge Dataset and Data Dictionary/heloc_dataset_v1_pure.csv")
feature_names = list(df)
data = df.values

y_original= data[:,:1]
X_original = data[:,1:]

In [3]:
# --- Basic Functions for Matrix editing --- 

def remove_row_with_all_same_val(data, target, val):
    row_no = 0 
    for row in data:
        for col in row:
            if (col == val):
                remove = True
            else:
                remove = False
                break

        if remove:
            data = np.delete(data, row_no, 0)
            target = np.delete(target, row_no, 0)     

        else:
            row_no += 1
    return data,target


def remove_row_with_vals(data, target, vals):
    row_no = 0 
    for row in data:
        for col in row:
            if (col in vals):
                data = np.delete(data, row_no, 0)
                target = np.delete(target, row_no, 0) 
                row_no -= 1
                break
        row_no += 1
    return data,target


def remove_col_with_vals(data, vals):
    no_cols = data.shape[1]
    no_rows = data.shape[0]
    row = 0
    while (no_rows > row):
        col = 0
        while (no_cols > col):
            if (data[row][col] in vals):
                data = np.delete(data, col, 1)
                no_cols -= 1
            else:
                col += 1
        row += 1     
    return data


def scaled_row(row):
    scld = []
    for k in range(features):
        scld.append((row[k] - scaler.mean_[k])/scaler.scale_[k])
    scld = np.array(scld)
    return scld
        
        
def masked_arr(A, mask):
    B = []
    for i in range(len(A)):
        row = []
        for j in range(len(A[0])):
            if mask[j] != 0:
                row.append(A[i][j])
        B.append(row)
    B = np.array(B)
    return B


def distance(row1, row2):
    dist = 0
    for i in range(len(row1)):
        t = (row1[i]-row2[i])**2
        dist += t
    dist = np.sqrt(dist)
    return dist

# predict features using kNN imputation
# need to test for both weighted and simple mean
# using 3 or 5 neighbors
def predict_feature_weighted(row, C, k, originalArr, ft_idx):
    
    distances = []
    for i in range(len(C)):
        distances.append(distance(row,C[i]))
    distances = np.array(distances)
    
    max_dist = np.max(distances)
        
    idx = np.argpartition(distances, k)

    values = []
    min_dists = []
    
    for i in range(k):
        values.append(originalArr[idx[i]][ft_idx])
#         print(Z[idx[i]])
        min_dists.append(distances[idx[i]])
    values = np.array(values) 
    min_dists = np.array(min_dists)

    weights = []
    for i in min_dists:
        weights.append(1 - (i/max_dist))
        
    imputed_val = 0
    for i in range(len(weights)):
        imputed_val += weights[i] * values[i]
        
    return imputed_val    


def predict_feature_mean(row, C, k, originalArr, ft_idx):
    
    distances = []
    for i in range(len(C)):
        distances.append(distance(row,C[i]))
    distances = np.array(distances)
        
    idx = np.argpartition(distances, k)
    
    values = []
    min_dists = []
    for i in range(k):
        values.append(originalArr[idx[i]][ft_idx])
#         print(Z[idx[i]])
        min_dists.append(distances[idx[i]])
    values = np.array(values) 
    min_dists = np.array(min_dists)
        
    imputed_val = 0
    for i in range(len(values)):
        imputed_val += values[i]/(len(values))
        
    return imputed_val

In [4]:
# --- Removing the rows with only -9 values ---
X, y = remove_row_with_all_same_val(X_original,y_original,-9)

# --- Testing ---
print(X_original.shape)
print(X.shape)
print(y_original.shape)
print(y.shape)

(10459, 23)
(9871, 23)
(10459, 1)
(9871, 1)


In [5]:
# --- Taking the array without any negatives present ---

samples, features = X.shape

X_good = []

for i in range(samples):
    remove = False
    for j in range(features):
        if X[i][j] < 0:
            remove = True
            break
    if remove == False:
        X_good.append(X[i])
        
X_good = np.array(X_good)

In [6]:
# --- Replacing -8 with k-nearest neighbours (average) ---

scaler = StandardScaler()
X_good_std = scaler.fit_transform(X_good)

X_no_8 = np.copy(X)

kNN = 5

cols_with_8 = [1,8,14,17,18,19,20,21,22]

for q in cols_with_8:

    print(q)
    
    for i in range(samples):

        if X[i][q] == -8:

            row_to_comp = []
            mask = []
            scaled = scaled_row(X[i])
            for j in range(features):
                if X[i][j] >= 0:
                    mask.append(1)
                    row_to_comp.append(scaled[j])
                else:
                    mask.append(0)
            row_to_comp = np.array(row_to_comp)
            mask = np.array(mask)

            S = masked_arr(X_good_std, mask)

            imputed = predict_feature_mean(row_to_comp, S, k, X_good_std, q)

            X_no_8[i][q] = imputed*scaler.scale_[q] + scaler.mean_[q]

/Users/steffen/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


1


8


14


17


18


19


20


21


22




In [8]:
# --- Selecting the column with -9  ---
y_nine = X_no_8[:,:1]

[[55]
 [61]
 [67]
 ..., 
 [74]
 [72]
 [66]]


In [ ]:
# --- Selecting the columns with -7  ---
X_no8 = np.copy(imputed_8_Y)

In [ ]:
# # --- Removing the columns that have -8 values ---
# X2 = remove_col_with_vals(X,[-8])

# # --- Testing ---
# print(X.shape)
# print(X2.shape)

In [ ]:
def test_model_degrees(X_tr,y_tr,degrees):
    RSS_list = []
    for deg in degrees:
        model = poly.polyfit(X_tr,y_tr,degree)
        pred = poly.polyval(X_tr, model)
        RSS = np.mean((Y_tr-pred)**2)
        RSS_list.append(RSS)
        
        
    plt.figure()
    plt.plot(degrees,RSS_list,'.-',color='r') 
    plt.xlabel('Degree')                            
    plt.ylabel('Performance')
    

def predict_values_poly_reg(X_tr,y_tr,X_test,deg):
    model = poly.polyfit(X_tr,y_tr,degree)
    pred = poly.polyval(X_test, model)
    return pred

def predict_values_lin_reg(X_tr,y_tr,X_test):
    model = linear_model.LinearRegression()
    model.fit(X_tr, y_tr)
    pred = model.predict(X_test)
    return pred
    

In [ ]:
# --- Shift categorical features so they match  ---

# Assumes array without labels where MaxDelq2PublicRecLast12M is in [9] and MaxDelqEver is in [10]

# [9]:
#     5,6 -> 6
#     7 -> 5
#     8,9 ->7

# [10]:
#     1-> 7
#     2 -> 0
#     3 -> 1
#     4 -> 2
#     5 -> 3
#     6 -> 4
#     7 -> 6
#     8 -> 5
#     9 -> 7

def shift_categorical (X):
    
    new_9 = []
    for i in X[9]:
        if i == 5:
            new_9.append(6)
        elif i == 7:
            new_9.append(5)
        elif i in [8,9]:
            new_9.append(7)
            
    new_10 = []
    for i in X[10]:
        if i == 1:
            new_10.append(7)
        elif i in [2,3,4,5,6,9]:
            new_10.append(i-2)
        elif i == 7:
            new_10.append(6)
        elif i == 8:
            new_10.append(5)
            
    X_shifted = np.copy(X)
    
    for i in range(len(X[0])):
        X_shifted[9][i] = new_9[i]
        X_shifted[10][i] = new_10[i]
        
    return X_shifted